In [1]:
# kaggle_clean_eda/
# ├── domain/
# │   ├── entities/
# │   │   └── dataset.py        # Entidades de dados
# │   └── interfaces/
# │       └── repositories.py   # Interfaces para acesso a dados
# ├── application/
# │   └── use_cases/
# │       ├── auth_kaggle.py    # Caso de uso: Autenticação
# │       └── eda_report.py     # Caso de uso: Geração de relatório
# ├── adapters/
# │   ├── repositories/
# │   │   └── kaggle_repo.py    # Implementação concreta da Kaggle API
# │   └── report_generators/
# │       └── pandas_eda.py     # Geração de relatórios EDA
# ├── infrastructure/
# │   ├── logging/
# │   │   └── logging_config.py # Configuração de logs
# │   └── file_handlers/
# │       └── csv_loader.py     # Carregamento de arquivos
# └── main.py                   # Ponto de entrada

### Importar as bibliotecas utilizadas no código

In [ ]:
import logging
import pandas as pd
from abc import ABC, abstractmethod
from kaggle.api.kaggle_api_extended import KaggleApi
from domain.interfaces.repositories import IKaggleRepository
from infrastructure.logging.logging_config import setup_logging
from application.use_cases.eda_report import GenerateEDAReportUseCase
from application.use_cases.auth_kaggle import AuthenticateKaggleUseCase

### Domain Layer (Core): Define as entidades e interfaces, o domínio da aplicação.

In [5]:
# domain/entities/dataset.py
class Dataset:
    def __init__(self, name: str, data: 'pd.DataFrame'):
        self.name = name
        self.data = data

# domain/interfaces/repositories.py

class IKaggleRepository(ABC):
    @abstractmethod
    def authenticate(self):
        pass
    
    @abstractmethod
    def download_dataset(self, dataset_name: str, path: str):
        pass

### Application Layer (Casos de Uso): Orquestrador da lógica/regras de negócio

ModuleNotFoundError: No module named 'domain'

In [ ]:
# application/use_cases/auth_kaggle.py

class AuthenticateKaggleUseCase:
    def __init__(self, repo: IKaggleRepository):
        self.repo = repo
    
    def execute(self):
        self.repo.authenticate()

### Adapters Layer (Implementações Concretas): Integração com serviços externos

In [ ]:
# application/use_cases/eda_report.py

class GenerateEDAReportUseCase:
    def __init__(self, repo: IKaggleRepository):
        self.repo = repo
    
    def execute(self, dataset_name: str, file_name: str):
        # Download do dataset
        self.repo.download_dataset(dataset_name, "./data")
        
        # Carregar dados
        df = pd.read_csv(f"./data/{file_name}")
        
        # Análise automatizada
        return {
            "info": df.info(),
            "summary": df.describe(include="object"),
            "missing_values": df.isnull().sum()
        }

### Infrastructure Layer (Detalhes Técnicos): Autenticação, definição da log.

In [ ]:
# adapters/repositories/kaggle_repo.py

class KaggleRepository(IKaggleRepository):
    def __init__(self):
        self.api = KaggleApi()
    
    def authenticate(self):
        self.api.authenticate()
    
    def download_dataset(self, dataset_name: str, path: str):
        self.api.dataset_download_files(dataset_name, path=path, unzip=True)

# infrastructure/logging/logging_config.py

def setup_logging():
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

### Função principal

In [ ]:
def main():
    setup_logging()
    
    # Configurações
    dataset_name = "flynn28/european-football-matches"
    file_name = "English Premier League.csv"
    
    # Inicializar repositório
    kaggle_repo = KaggleRepository()
    
    # Autenticar
    auth_use_case = AuthenticateKaggleUseCase(kaggle_repo)
    auth_use_case.execute()
    
    # Gerar relatório EDA
    eda_use_case = GenerateEDAReportUseCase(kaggle_repo)
    report = eda_use_case.execute(dataset_name, file_name)
    
    # Exibir resultados
    print("Resumo do Dataset:")
    print(report["summary"])
    print("\nValores Ausentes:")
    print(report["missing_values"])

if __name__ == "__main__":
    main()